In [1]:
import bagpy
from bagpy import bagreader
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import rosbag
import os

# from geometry_msgs.msg import Point, Quaternion, Vector3
# from snapstack_msgs.msg import State, Goal, AttitudeCommand, ControlLog, QuadFlightMode

plt.rcParams.update({'font.size': 24})

In [2]:
# filename = 'sim/pid_figure_eight.bag'
# filename = 'sim/pid_constant_wind_6-12.bag'
# filename = 'sim/coml_spline_wind_6-12.bag'
# filename = 'sim/reg_P_2e-3_reg_k_R_2e-3_k_R_z_1_wind_6-12.bag'
# filename = 'sim/reg_P_1e-1_reg_k_R_0_wind_6-12.bag'

# filename = 'sim/coml_spline_wind_0-6.bag'
# filename = 'sim/pid_spline_wind_0-6.bag'

# filename = 'sim/coml_spline_wind_0.bag'
# filename = 'sim/pid_spline_wind_0.bag'

# filename = 'sim/coml_figure_eight_wind_0.bag'
# filename = 'sim/pid_figure_eight_wind_0.bag'

# filename = 'sim/coml_circle_wind_0.bag'
# filename = 'sim/pid_circle_wind_0.bag'

# filename = 'sim/pid_figure_eight_wind_10.bag'
# filename = 'sim/coml_figure_eight_wind_10.bag'

# filename = 'sim/coml_circle_wind_10.bag'
# filename = 'sim/pid_circle_wind_10.bag'

# filename = 'sim/pid_circle_wind_sine.bag'
# filename = 'sim/coml_circle_wind_sine.bag'

# filename = 'sim/pid_figure_eight_wind_sine.bag'     
# filename = 'sim/coml_figure_eight_wind_sine.bag'

# filename = "sim/_2024-07-09-14-27-12.bag"
# filename = "sim/_2024-07-09-15-56-55.bag"

# filename = "sim/_2024-07-22-22-19-41.bag"
# # filename = "sim/_2024-07-22-22-27-56.bag"

# b = bagreader(f'../rosbags/{filename}')

# topic_log = b.message_by_topic('/HX14s/log')
# topic_wind = b.message_by_topic('/HX14s/wind')

# # df_wind = pd.read_csv(topic_wind)
# df_log = pd.read_csv(topic_log)
# print(df_log.keys())
# # print(df_wind.keys())

filename = "sim/_2024-07-22-22-19-41.bag"
# filename = "sim/_2024-07-22-22-27-56.bag"

rosbag_dir = "../rosbags/sim"

results_dict = {}

for subdir, dirs, files in os.walk(rosbag_dir):
    for filename in files:
        if filename[-3:] != 'bag':
            continue

        b = bagreader(f'../rosbags/sim/{filename}')

        topic_log = b.message_by_topic('/HX14s/log')
        topic_wind = b.message_by_topic('/HX14s/wind')

        # df_wind = pd.read_csv(topic_wind)
        df_log = pd.read_csv(topic_log)
        # print(df_log.keys())
        # print(df_wind.keys())

        err_x = df_log['p_err.x'].to_numpy()
        err_y = df_log['p_err.y'].to_numpy()
        err_z = df_log['p_err.z'].to_numpy()
        total_err = np.vstack((err_x, err_y, err_z)).T # T x 3
        avg_tracking_error = np.mean(np.linalg.norm(total_err, axis=1))

        results_dict[filename] = avg_tracking_error

        df_log['real_time'] = df_log['Time'] - df_log['Time'][0]
        # df_wind['real_time'] = df_wind['Time'] - df_wind['Time'][0]

        df_log['real_time'][len(df_log['real_time'])-1]

sorted_dict = sorted( ((v,k) for k,v in results_dict.items()), reverse=True)
for v, k in sorted_dict:
    print(f"Filename: {k}")
    print(f"\tAvg Tracking Error: {v}\n")

[INFO]  Data folder ../rosbags/sim/seed=0_M=50_E=2500_pinit=2.00_pfreq=2500_regP=100.0000_spline already exists. Not creating.
No data on the topic:/HX14s/wind
[INFO]  Data folder ../rosbags/sim/seed=0_M=50_E=2500_pinit=2.00_pfreq=2500_regP=100.0000_figure_eight already exists. Not creating.
No data on the topic:/HX14s/wind
[INFO]  Successfully created the data folder ../rosbags/sim/seed=0_M=50_E=2500_pinit=2.00_pfreq=2500_regP=100.0000_spline_wind_12.
No data on the topic:/HX14s/wind
[INFO]  Successfully created the data folder ../rosbags/sim/seed=0_M=50_E=2500_pinit=2.00_pfreq=2500_regP=250.0000_spline_wind_9.
No data on the topic:/HX14s/wind
[INFO]  Data folder ../rosbags/sim/seed=0_M=50_E=2500_pinit=2.50_pfreq=25_regP=1.0000_spline already exists. Not creating.
No data on the topic:/HX14s/wind
[INFO]  Data folder ../rosbags/sim/seed=0_M=50_E=3500_pinit=3.00_pfreq=25_regP=0.0050_figure_eight already exists. Not creating.
No data on the topic:/HX14s/wind
[INFO]  Data folder ../rosbag

In [3]:
err_x = df_log['p_err.x'].to_numpy()
err_y = df_log['p_err.y'].to_numpy()
err_z = df_log['p_err.z'].to_numpy()
total_err = np.vstack((err_x, err_y, err_z)).T # T x 3
avg_tracking_error = np.mean(np.linalg.norm(total_err, axis=1))
print('avg tracking error = ', avg_tracking_error)

df_log['real_time'] = df_log['Time'] - df_log['Time'][0]
# df_wind['real_time'] = df_wind['Time'] - df_wind['Time'][0]

df_log['real_time'][len(df_log['real_time'])-1]

# print(df_log['real_time'][len(df_log['real_time'])-1])
# print(df_log['p_err.x'][len(df_log['real_time'])-1])
# print(df_log['p_ref.x'][len(df_log['real_time'])-1] - df_log['p.x'][len(df_log['real_time'])-1])

avg tracking error =  0.06216566376576116


34.12036681175232

In [4]:
def plot(df_log, df_wind, xlim_left=None, xlim_right=None):
    # A, y
    fig, axes = plt.subplots(1, 2, figsize=(30, 10))

    df_log.plot(x='real_time', y='A_norm', ax=axes[0])
    axes[0].set_ylabel('A_norm')
    axes[0].set_xlabel('t')
    axes[0].set_xlim(left=xlim_left, right=xlim_right)
    axes[0].legend()

    df_log.plot(x='real_time', y='y_norm', ax=axes[1])
    axes[1].set_ylabel('y_norm')
    axes[1].set_xlabel('t')
    axes[1].legend()
    axes[1].set_xlim(left=xlim_left, right=xlim_right)

    fig.suptitle('A, y')
    plt.tight_layout()
    plt.show()

    # Position
    fig, axes =plt.subplots(1, 3, figsize=(30, 10))

    df_log.plot(x='real_time', y='p.x', ax=axes[0], style='r-', label='q')
    df_log.plot(x='real_time', y='p_ref.x', ax=axes[0], style='b--', label='r')
    axes[0].set_ylabel('x')
    axes[0].set_xlabel('t')
    axes[0].set_xlim(left=xlim_left, right=xlim_right)
    axes[0].legend()

    df_log.plot(x='real_time', y='p.y', ax=axes[1], style='r-', label='q')
    df_log.plot(x='real_time', y='p_ref.y', ax=axes[1], style='b--', label='r')
    axes[1].set_ylabel('y')
    axes[1].set_xlabel('t')
    axes[1].set_xlim(left=xlim_left, right=xlim_right)
    axes[1].legend()

    df_log.plot(x='real_time', y='p.z', ax=axes[2], style='r-', label='q')
    df_log.plot(x='real_time', y='p_ref.z', ax=axes[2], style='b--', label='r')
    axes[2].set_ylabel('z')
    axes[2].set_xlabel('t')
    axes[2].set_xlim(left=xlim_left, right=xlim_right)
    axes[2].legend()

    fig.suptitle('Position')
    plt.tight_layout()
    plt.show()

    # Position
    fig, axes =plt.subplots(1, 3, figsize=(30, 10))

    df_log.plot(x='real_time', y='p.x', ax=axes[0], style='r-', label='q')
    df_log.plot(x='real_time', y='p_ref.x', ax=axes[0], style='b--', label='r')
    axes[0].set_ylabel('x')
    axes[0].set_xlabel('t')
    axes[0].set_xlim(left=0, right=60)
    axes[0].legend()

    df_log.plot(x='real_time', y='p.y', ax=axes[1], style='r-', label='q')
    df_log.plot(x='real_time', y='p_ref.y', ax=axes[1], style='b--', label='r')
    axes[1].set_ylabel('y')
    axes[1].set_xlabel('t')
    axes[1].set_xlim(left=0, right=60)
    axes[1].legend()

    df_log.plot(x='real_time', y='p.z', ax=axes[2], style='r-', label='q')
    df_log.plot(x='real_time', y='p_ref.z', ax=axes[2], style='b--', label='r')
    axes[2].set_ylabel('z')
    axes[2].set_xlabel('t')
    axes[2].set_xlim(left=0, right=60)
    axes[2].legend()

    fig.suptitle('Position')
    plt.tight_layout()
    plt.show()

    # Parametric
    plt.figure(figsize=(30, 30))

    plt.plot(df_log['p.x'], df_log['p.y'], 'r-', label='q')
    plt.plot(df_log['p_ref.x'], df_log['p_ref.y'], 'b--', label='r')
    plt.xlabel('x')
    plt.ylabel('y')
    
    plt.legend()
    plt.title('Parametric Position')
    plt.tight_layout()
    plt.show()

    # Velocity
    fig, axes = plt.subplots(1, 3, figsize=(30, 10))

    df_log.plot(x='real_time', y='v.x', ax=axes[0], style='r-', label='dq')
    df_log.plot(x='real_time', y='v_ref.x', ax=axes[0], style='b--',label='dr')
    axes[0].set_ylabel('x')
    axes[0].set_xlabel('t')
    axes[0].set_xlim(left=xlim_left, right=xlim_right)
    axes[0].legend()

    df_log.plot(x='real_time', y='v.y', ax=axes[1], style='r-', label='dq')
    df_log.plot(x='real_time', y='v_ref.y', ax=axes[1], style='b--',label='dr')
    axes[1].set_ylabel('y')
    axes[1].set_xlabel('t')
    axes[1].set_xlim(left=xlim_left, right=xlim_right)
    axes[1].legend()

    df_log.plot(x='real_time', y='v.z', ax=axes[2], style='r-', label='dq')
    df_log.plot(x='real_time', y='v_ref.z', ax=axes[2], style='b--',label='dr')
    axes[2].set_ylabel('z')
    axes[2].set_xlabel('t')
    axes[2].set_xlim(left=xlim_left, right=xlim_right)
    axes[2].legend()

    fig.suptitle('Velocity')
    plt.tight_layout()
    plt.show()

    # Attitude
    fig, axes = plt.subplots(1, 3, figsize=(30, 10))

    df_log.plot(x='real_time', y='rpy.x', ax=axes[0], style='r-', label='R')
    df_log.plot(x='real_time', y='rpy_ref.x', ax=axes[0], style='b--', label='R_d')
    axes[0].set_ylabel('x')
    axes[0].set_xlabel('t')
    axes[0].set_xlim(left=xlim_left, right=xlim_right)
    axes[0].legend()

    df_log.plot(x='real_time', y='rpy.y', ax=axes[1], style='r-', label='R')
    df_log.plot(x='real_time', y='rpy_ref.y', ax=axes[1], style='b--', label='R_d')
    axes[1].set_ylabel('y')
    axes[1].set_xlabel('t')
    axes[1].set_xlim(left=xlim_left, right=xlim_right)
    axes[1].legend()

    df_log.plot(x='real_time', y='rpy.z', ax=axes[2], style='r-', label='R')
    df_log.plot(x='real_time', y='rpy_ref.z', ax=axes[2], style='b--', label='R_d')
    axes[2].set_ylabel('z')
    axes[2].set_xlabel('t')
    axes[2].set_xlim(left=xlim_left, right=xlim_right)
    axes[2].legend()

    fig.suptitle('Attitude')
    plt.tight_layout()
    plt.show()

    # Angular velocity
    fig, axes = plt.subplots(1, 3, figsize=(30, 10))

    df_log.plot(x='real_time', y='w.x', ax=axes[0], style='r-', label='Omega')
    df_log.plot(x='real_time', y='w_ref.x', ax=axes[0], style='b--', label='Omega_d')
    axes[0].set_ylabel('x')
    axes[0].set_xlabel('t')
    axes[0].set_xlim(left=xlim_left, right=xlim_right)
    axes[0].legend()

    df_log.plot(x='real_time', y='w.y', ax=axes[1], style='r-', label='Omega')
    df_log.plot(x='real_time', y='w_ref.y', ax=axes[1], style='b--', label='Omega_d')
    axes[1].set_ylabel('y')
    axes[1].set_xlabel('t')
    axes[1].set_xlim(left=xlim_left, right=xlim_right)
    axes[1].legend()

    df_log.plot(x='real_time', y='w.z', ax=axes[2], style='r-', label='Omega')
    df_log.plot(x='real_time', y='w_ref.z', ax=axes[2], style='b--', label='Omega_d')
    axes[2].set_ylabel('z')
    axes[2].set_xlabel('t')
    axes[2].set_xlim(left=xlim_left, right=xlim_right)
    axes[2].legend()

    fig.suptitle('Angular velocity')
    plt.tight_layout()
    plt.show()

    # F_W
    fig, axes = plt.subplots(1, 3, figsize=(30, 10))

    df_log.plot(x='real_time', y='F_W.x', ax=axes[0], style='r-', label='F_W')
    axes[0].set_ylabel('x')
    axes[0].set_xlabel('t')
    axes[0].set_xlim(left=xlim_left, right=xlim_right)
    axes[0].legend()

    df_log.plot(x='real_time', y='F_W.y', ax=axes[1], style='r-', label='F_W')
    axes[1].set_ylabel('y')
    axes[1].set_xlabel('t')
    axes[1].legend()
    axes[1].set_xlim(left=xlim_left, right=xlim_right)

    df_log.plot(x='real_time', y='F_W.z', ax=axes[2], style='r-', label='F_W')
    axes[2].set_ylabel('z')
    axes[2].set_xlabel('t')
    axes[2].legend()

    fig.suptitle('F_W')
    axes[2].set_xlim(left=xlim_left, right=xlim_right)
    plt.tight_layout()
    plt.show()

    # f_hat
    fig, axes = plt.subplots(1, 3, figsize=(30, 10))

    df_wind['w_total.x'] = df_wind['w_nominal.x'] + df_wind['w_gust.x']
    df_wind['w_total.y'] = df_wind['w_nominal.y'] + df_wind['w_gust.y']
    df_wind['w_total.z'] = df_wind['w_nominal.z'] + df_wind['w_gust.z']

    df_log.plot(x='real_time', y='f_hat.x', ax=axes[0], style='r-', label='f_hat')
    df_wind.plot(x='real_time', y='w_total.x', ax=axes[0], style='b--', label='f_ext')
    axes[0].set_ylabel('x')
    axes[0].set_xlabel('t')
    axes[0].set_xlim(left=xlim_left, right=xlim_right)
    axes[0].legend()

    df_log.plot(x='real_time', y='f_hat.y', ax=axes[1], style='r-', label='f_hat')
    df_wind.plot(x='real_time', y='w_total.y', ax=axes[1], style='b--', label='f_ext')
    axes[1].set_ylabel('y')
    axes[1].set_xlabel('t')
    axes[1].legend()
    axes[1].set_xlim(left=xlim_left, right=xlim_right)

    df_log.plot(x='real_time', y='f_hat.z', ax=axes[2], style='r-', label='f_hat')
    df_wind.plot(x='real_time', y='w_total.z', ax=axes[2], style='b--', label='f_ext')
    axes[2].set_ylabel('z')
    axes[2].set_xlabel('t')
    axes[2].legend()

    fig.suptitle('f_hat')
    axes[2].set_xlim(left=xlim_left, right=xlim_right)
    plt.tight_layout()
    plt.show()

plot(df_log, df_wind)


NameError: name 'df_wind' is not defined